In [1]:
import sqlite3
import pandas as pd
import os
import copy
pd.options.mode.chained_assignment = None

In [2]:
process_path = os.path.join(os.sep+"home"+os.sep+"jupyter"+os.sep+"Team-Prophecy","Data","02_processed","intermediate.db")
print(process_path)

/home/jupyter/Team-Prophecy/Data/02_processed/intermediate.db


In [3]:
process_connection = sqlite3.connect(process_path)

In [4]:
inputs = {    
    'prog_desc'     : "All",
    'courses'       : "All",
    'mod'           : "F2F",
    'visa'          : "F1 Visa",
    'ExpN_eat'      : 150
}

In [5]:
# Building the model parameters
model_params = {
    'prog_desc'         : inputs['prog_desc'],
    'courses'           : inputs['courses'],
    'mod'               : inputs['mod'],
    'visa'              : inputs['visa'],
    'ExpN_eat'          : inputs['ExpN_eat'],
    'enr_hist'          : 4 if inputs['visa'] == 'F1 Visa' else 8,
    'filters'           : {
        'prog_filter'   : inputs['prog_desc'] != 'All',
        'course_filter' : inputs['courses'] != 'All',
        'visa_filter'   : inputs['visa'] != 'combined'
    },
    'groupby'           : {
        'mod_groupby'   : inputs['mod'] != 'combined'
    }
}

In [6]:
def list_to_str(l):
    """
    Create a string in the format of "('a', 'b', 'c', 'd')" from list ['a', 'b', 'c', 'd']
    """
    return str(tuple(l))

In [7]:
student_stat_query = """
    SELECT rs.reg_term_code, rs.reg_stu_id, crs, rs.sect_id, rs.reg_new_ret_stu, rs.reg_final_status, scoff.cum_seat_total, scoff.cum_seat_enroll, scoff.cum_seat_avail
    FROM registration_status rs
    INNER JOIN (SELECT sc.cum_term, i_n.instr_home_org, sc.cum_instr, sc.cum_term_code, sc.cum_sect_id, 
        sc.cum_seat_enroll+abs(sc.cum_seat_avail)+sc.cum_seat_wait as cum_seat_total,
        IIF(sc.cum_seat_avail < 0, sc.cum_seat_enroll+abs(sc.cum_seat_avail),sc.cum_seat_enroll) as cum_seat_enroll,
        IIF(sc.cum_seat_avail < 0, 0, sc.cum_seat_avail) as cum_seat_avail
    FROM semester_course_offerings sc
    INNER JOIN instructors i_n ON sc.cum_instr == i_n.instr_name) scoff ON 
        scoff.cum_term == rs.reg_term_code
        AND scoff.cum_term_code == rs.crs
        AND scoff.cum_sect_id == rs.sect_id
    WHERE 1=1 
"""
student_details_query = """
    SELECT stu_admit_term_code, stu_college, stu_deg_level, stu_dept, stu_id, stu_res, stu_prog, stu_visa, stu_bam
    FROM student_details 
    WHERE 1=1 
"""

program_course_offerings = """
    SELECT * FROM PROGRAM_COURSE_OFFERINGS 
"""

# Program Filter
if model_params['filters']['prog_filter']:
    student_details_query += f"AND stu_prog = '{model_params['prog_desc']}' "

# Course Filter
if model_params['filters']['course_filter']:
    student_stat_query += f"AND crs IN {list_to_str(model_params['courses'])} "

# Visa Filter
if model_params['filters']['visa_filter']:
    student_details_query += f"AND stu_visa = '{model_params['visa']}' "

In [8]:
student_stat = pd.DataFrame(process_connection.execute(student_stat_query).fetchall(),columns=["reg_term_code", "student_id", "crs", "sect_id", "returning_student", "reg_status",
                                                                                              "cum_total_enrollment","cum_seat_enrollment","cum_seat_available"])

#WHERE reg_final_status IN ('W','R')
#GROUP BY reg_term_code, crs, sect_id, reg_final_status

student_details = pd.DataFrame(process_connection.execute(student_details_query).fetchall(),columns=["reg_term_code","stu_college","stu_deg_level","stu_dept","student_id",
                                                                                                     "stu_res","stu_prog","stu_visa","stu_bam"])

pco_df = pd.DataFrame(process_connection.execute(program_course_offerings),columns=["stu_prog","crs","required"])

total_stat = student_stat.merge(student_details, on=["reg_term_code","student_id"], how="inner").fillna(0)
total_stat = total_stat.merge(pco_df, on=["stu_prog","crs"])

In [9]:
total_stat = total_stat.loc[total_stat["required"] == 1,:]

In [10]:
total_student_population = total_stat[["reg_term_code"]].groupby("reg_term_code").count().reset_index()

In [11]:
reg_term_values = dict(total_stat.groupby(["reg_term_code"])["reg_term_code"].count())

In [12]:
#program_limit_values = dict(total_stat.groupby(["stu_prog"])["stu_prog"].count())

In [13]:
total_courses = total_stat[["reg_term_code","stu_prog","crs","sect_id","reg_status","cum_total_enrollment","cum_seat_enrollment","cum_seat_available"]]

REF Courses is just a fancy way of determining how, from a professors point of view, courses are included.

In [14]:
ref_course = total_courses[["reg_term_code","crs","sect_id","cum_total_enrollment","cum_seat_enrollment","cum_seat_available"]]

In [15]:
registered_courses = total_courses.loc[total_courses["reg_status"] == "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
#nonregistered_courses = total_courses.loc[total_courses["reg_status"] != "R",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

Keep in mind: Our registered values will be dependent on what's in our subset.

In [16]:
req_courses_context = registered_courses.merge(ref_course[["reg_term_code", "crs","sect_id", "cum_total_enrollment"]], on=["reg_term_code", "crs","sect_id"], how="inner").drop_duplicates()

In [17]:
req_courses_context.columns

Index(['reg_term_code', 'stu_prog', 'crs', 'sect_id', 'reg_status',
       'cum_total_enrollment'],
      dtype='object')

In [18]:
req_courses_context.loc[req_courses_context["cum_total_enrollment"] < req_courses_context["reg_status"],["cum_total_enrollment"]]  = req_courses_context["reg_status"]

In [19]:
reg_term_courses_context = req_courses_context.groupby(["reg_term_code"])

In [20]:
req_courses_context

,reg_term_code,stu_prog,crs,sect_id,reg_status,cum_total_enrollment
0,202010,EC-MS-AIT,AIT524,003,2,25
2,202010,EC-MS-AIT,AIT542,002,2,20
4,202010,EC-MS-AIT,AIT602,001,1,25
5,202010,EC-MS-AIT,AIT673,DL1,1,25
6,202010,EC-MS-AIT,AIT677,001,3,25
...,...,...,...,...,...,...
1544,202310,EC-MS-CEIE,CEIE767,001,2,15
1546,202310,EC-MS-CEIE,CEIE795,001,55,60
1601,202310,EC-MS-DAEN,INFS580,002,2,4
1603,202310,EC-MS-DAEN,INFS580,003,1,12


In [21]:
from ortools.linear_solver import pywraplp
from itertools import chain

For each semester's influx of students, we want the following:
- How many butts are going to be in a seat for a given course section
- How many butts are going to be waitlisted for a given course section

**DECISION VARIABLES**

X = Number of Students who applied

W = Whether the student was waitlisted for the class section

A = Whether the student has already registered for the class

B = Whether the student has already waitlisted for the class

**Indices**

$s\in SemesterProgram$ = [1..N]

$c\in Course$ = [1..N]

$z\in Course Section$ = [1..N]

$d\in Student ID$ = [1..N]

**Indexed Variables**

$CAP_s$ = Total number of students within the semester's program that MUST find a class

$PCS_{sc}$ = The ratio of students expected to take this course. _This is based on what our semester's number of registered students were previously_

$O_{sc}$ = The MAXIMUM number of students for a course

$P_{scz}$ = The MINIMUM number of classes a student must take

**Decision Variables**

$ X_{sczd} $ = Students who successfully registered

$ W_{sczd} $ = Students who were waitlisted


$ A_{sc} $ = This counts the total number of students who registered for a class

$ B_{sc} $ = This counts the total number of students who were waitlisted for a class


$ S_{scd} = \left\{
    \begin{array}\\
        1 & \mbox{if the student hasn't taken the course} \  \\
        0 & \mbox{else}
    \end{array}
\right. $

**Optimization Function**

$$ \text{Min} \sum_{\substack{s\in Semester\\c\in Course\\z\in Course Section\\d\in StudentID}}^{} \text{W_s_c_z_d} $$


**Constraints**

This weights students based on who is in the class. _This is effectively a flow diagram_:

$$ PCS_{cs} * A_{sc}  <= O_{sc} + B_{sczd} $$

We must have a maximum number of classes the student takes in place. _This helps ensure diversity_:

$$ \sum_{\substack{s\in Semester\\c\in Course\\z\in Course Section}}^{} X_{sczd} <= O_{sc} \text{ ; } \forall{d\in StudentID} $$

We must have a minimmum number of classes the student takes in place. _This helps ensure diversity_:

$$ \sum_{\substack{s\in Semester\\c\in Course\\z\in Course Section}}^{} X_{sczd} >= P_{scz} \text{ ; } \forall{d\in StudentID}  $$

If only $ X_{sczd} $ or $ W_{sczd} $ can exist at any given time. _They cannot both be used_:

$$ X_{sczd} + W_{sczd} <= 1 \text{ ; } \forall{s\in Semester\\c\in Course\\z\in Course Section \\d\in StudentID} $$

Students can only take ONE class per class section:

$$ \sum_{\substack{s\in Semester\\z\in Course Section}}^{} X_{sczd} <= 1 \text{ ; } \forall{c\in Course \\d\in StudentID} $$

$$ \sum_{\substack{s\in Semester\\z\in Course Section}}^{} W_{sczd} <= 1 \text{ ; } \forall{c\in Course \\d\in StudentID} $$

This maps all students under a registered class to our $ A_{sc} $ and $ B_{sc} $ values effectively:

$$ \sum_{\substack{z\in Course Section\\d\in StudentID}}^{} X_{sczd} =  A_{sc} \text{ ; } \forall{z\in Course Section \\d\in StudentID} $$

$$ \sum_{\substack{z\in Course Section\\d\in StudentID}}^{} W_{sczd} = B_{sc} \text{ ; } \forall{z\in Course Section \\d\in StudentID} $$

Our $ X_{sczd} $ or $ W_{sczd} $ values will be binary:

$$ X_{sczd} = \left\{
    \begin{array}\\
        1 & \mbox{if the student is registered} \  \\
        0 & \mbox{else}
    \end{array}
\right. $$

$$ W_{sczd} = \left\{
    \begin{array}\\
        1 & \mbox{if the student is waitlisted} \  \\
        0 & \mbox{else}
    \end{array}
\right. $$

**Indexed Variables for us (when we have time)**

$J_{sc}$ = Max Credits a student can apply for (Replaces max classes)

$R_{scz}$ = MAX number of students for a course section

$K_{sc}$ = Min Credits a student can apply for (Replaces min classes)

What we're going to do
-> One thing will be a script file that runs automatically behind the scenes
-> In the frontend, what we're going to do is define an input dictionary
    This is the code that we want: Only international students, and we'll have 

In [22]:
all_semesters = list(reg_term_values.keys())
#program_limit_values <- included because we need a way to constraint students
all_pcs_list = list(req_courses_context[["stu_prog","crs","sect_id"]].drop_duplicates().itertuples(index=False,name=None))
all_pcs_dict = {}
#References what program contains which class
pc_dict = {}
cs_dict = {}
for t_pcs in all_pcs_list:
    if t_pcs[0] not in all_pcs_dict:
        pc_dict[t_pcs[0]] = []
        all_pcs_dict[t_pcs[0]] = {}
    pc_dict[t_pcs[0]].append(t_pcs[1])
    if t_pcs[1] not in all_pcs_dict[t_pcs[0]]:
        cs_dict[t_pcs[1]] = []
        all_pcs_dict[t_pcs[0]][t_pcs[1]] = []
    cs_dict[t_pcs[1]].append(t_pcs[2])
    all_pcs_dict[t_pcs[0]][t_pcs[1]].append(t_pcs[2])

In [23]:
req_courses_context
#req_courses_context[["reg_term_code","stu_prog","crs","sect_id","reg_status"]]

,reg_term_code,stu_prog,crs,sect_id,reg_status,cum_total_enrollment
0,202010,EC-MS-AIT,AIT524,003,2,25
2,202010,EC-MS-AIT,AIT542,002,2,20
4,202010,EC-MS-AIT,AIT602,001,1,25
5,202010,EC-MS-AIT,AIT673,DL1,1,25
6,202010,EC-MS-AIT,AIT677,001,3,25
...,...,...,...,...,...,...
1544,202310,EC-MS-CEIE,CEIE767,001,2,15
1546,202310,EC-MS-CEIE,CEIE795,001,55,60
1601,202310,EC-MS-DAEN,INFS580,002,2,4
1603,202310,EC-MS-DAEN,INFS580,003,1,12


In [24]:
req_courses_context["stu_prog"].drop_duplicates().tolist()

['EC-MS-AIT', 'EC-MS-CEIE', 'EC-MS-DAEN', 'EC-MS-ISYS']

In [25]:
#Constraints
#total_student_population
#program_limit_values

Fall 2020
150 -> 80 Program_1, 40 Program_2, 30 Program_3
50
1 student -> 15 credits = 3-4 classes

In [26]:
for a_sem in all_semesters:
    cxt_grp = reg_term_courses_context.get_group(a_sem)
    #WE HAVE RATIOS HERE IN CASE WE NEED TO DO MORE ANALYSIS OR MATCH IT TO 1
    #
    #These will be our constraints
    total_capacity = cxt_grp[["crs","reg_status"]].set_index("crs").to_dict()["reg_status"]
    
    cxt_grp["reg_status"] = cxt_grp["reg_status"]/reg_term_values[a_sem]
    #total_students_list = cxt_grp["cum_total_enrollment"]/reg_term_values[a_sem]
    pcs_weighting_ratio = cxt_grp[["reg_term_code","stu_prog","crs","sect_id","reg_status"]].set_index(["crs"])["reg_status"].to_dict()
    pcs_weighting_ratio = {k : v for k,v in pcs_weighting_ratio.items()} #"("+",".join(k)+")"
    #Now that we have the ratios, we can begin with the objective function.
    #
    decision_vars = {}
    const_vars = {}
    
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        print("Cannot get solver")
        continue
        
    
    program_l = cxt_grp["stu_prog"].drop_duplicates().tolist()
    cs_l = cxt_grp["crs"].drop_duplicates().tolist()

    #These will be our reference variables
    program_ref = {}
    course_ref = {}
    
    program_w_ref = {}
    course_w_ref = {}
    course_w_ref = {}
    
    course_student_ref = {}
    course_student_w_ref = {}
    
    #Student ID
    x_decision_vars = {}
    w_decision_vars = {}
    a_decision_vars = {}
    b_decision_vars = {}
    #
    s_inf = solver.infinity()
    
    all_pcs = list(cxt_grp[["stu_prog","crs"]].drop_duplicates().itertuples(index=False,name=None))
    for a_pc in all_pcs:
        if a_pc[0] not in program_ref.keys():
            program_ref[a_pc[0]] = []
            program_w_ref[a_pc[0]] = []
        
        if a_pc[1] not in course_ref.keys():
            course_ref[a_pc[1]] = []
            course_w_ref[a_pc[1]] = []
            
        if a_pc[1] not in course_student_ref.keys():
            course_student_ref[a_pc[1]] = {}
            course_student_w_ref[a_pc[1]] = {}
        
        for student_id in range(1,reg_term_values[a_sem]):
            #We'll have two primary decision variables: Waitlist and Student
            #This will be a minimization function for our purposes, as we want all students
            # to be included.
            
            x_name = f"x({student_id},{a_sem})"
            w_name = f"w({student_id},{a_sem})"
            
            #Declares the decision variable
            
            
            decision_vars[x_name] = solver.IntVar(0,1,x_name)
            decision_vars[w_name] = solver.IntVar(0,1,w_name)
            
            
            #waitlist_decision_vars[student_id] = pcs_weighting_ratio[f"({a_sem},{a_pc[0]},{a_pc[1]},{a_pc[2]})"]
            if student_id not in x_decision_vars:
                x_decision_vars[student_id] = []
            if student_id not in w_decision_vars:
                w_decision_vars[student_id] = []
            x_decision_vars[student_id].append(decision_vars[x_name])
            w_decision_vars[student_id].append(decision_vars[w_name])
            
            program_ref[a_pc[0]].append(decision_vars[x_name])
            program_w_ref[a_pc[0]].append(decision_vars[w_name])
            
            
            course_ref[a_pc[1]].append(decision_vars[x_name])
            course_w_ref[a_pc[1]].append(decision_vars[w_name])
            
            course_student_ref[a_pc[1]][student_id] = decision_vars[x_name]
            course_student_w_ref[a_pc[1]][student_id] = decision_vars[w_name]
    
    
    #for p in program_l:
    #    const_vars[f"01_program_{p}_constr"] = solver.Add(solver.Sum(program_ref[p]) + solver.Sum(program_w_ref[p])
    #                                                      >= program_limit_values[p],name=f"01_program_{p}_constr")
    # A student can't take more than 15 credits of a course
    for cs in cs_l:
        const_vars[f"01_course_with_w_{cs}_constr_ratio"] = solver.Add(pcs_weighting_ratio[cs]*solver.Sum(course_ref[cs]) + solver.Sum(course_w_ref[cs]) >= total_capacity[cs], name=f"01_course_with_w_{cs}_constr_ratio")
        const_vars[f"01_course_{cs}_constr"] = solver.Add(solver.Sum(course_ref[cs]) <= total_capacity[cs], name=f"02_course_with_w_{cs}_constr")
        for student_id in range(1,reg_term_values[a_sem]): 
            const_vars[f"02_course_{cs}_sections_waitlist_constr"] = solver.Add(course_student_w_ref[cs][student_id] <= 1, name=f"03_course_{cs}_sections_waitlist_constr")
            const_vars[f"02_course_{cs}_sections_constr"] = solver.Add(course_student_ref[cs][student_id] <= 1, name=f"03_course_{cs}_sections_constr")
            
            const_vars[f"03_one_regstat_{student_id}"] = solver.Add(course_student_ref[cs][student_id] + course_student_w_ref[cs][student_id] == 1, name=f"04_one_regstat_{student_id}")
            
            a_name = f"a({cs},{a_sem})"
            b_name = f"b({cs},{a_sem})"
            a_decision_vars[a_name] = solver.IntVar(0,s_inf,a_name)
            b_decision_vars[b_name] = solver.IntVar(0,s_inf,b_name)
            
            const_vars[f"03_student_bounds_{student_id}"] = solver.Add(solver.Sum(x_decision_vars[student_id]) <= 3, name=f"03_student_bounds_{student_id}")
            #const_vars[f"03_student_w_bounds_{student_id}"] = solver.Add(solver.Sum(w_decision_vars[student_id]) <= 3, name=f"03_student_bounds_w_{student_id}")
        const_vars[f"04_organization_course_{cs}_const"] = solver.Add(solver.Sum([course_student_ref[cs][student_id] for student_id in range(1,reg_term_values[a_sem])]) == a_decision_vars[a_name],name=f"04_organization_course_{cs}_const")
        const_vars[f"04_organization_wait_course_{cs}_const"] = solver.Add(solver.Sum([course_student_w_ref[cs][student_id] for student_id in range(1,reg_term_values[a_sem])]) == b_decision_vars[b_name],name=f"04_organization_wait_course_{cs}_const")
        
    
    # Minimization function 
    # -> For later * a_decision_vars[student_id]
    
    solver.Minimize(solver.Sum([solver.Sum(w_decision_vars[student_id])
                                for student_id in range(1,reg_term_values[a_sem])]))
    solver.Solve()
    break

In [27]:
solver.NumVariables()

2744

In [28]:
solver.NumConstraints()

2800

In [29]:
print('Objective value =', solver.Objective().Value())

Objective value = 636.0


In [496]:
#for i in list(x_decision_vars.values()):
#    print(f"{i}")

In [30]:
for i in list(a_decision_vars.values()):
    print(f"{i.name()} -> {i.SolutionValue()}")

a(AIT524,202010) -> 2.0
a(AIT542,202010) -> 2.0
a(AIT602,202010) -> 1.0
a(AIT673,202010) -> 1.0
a(AIT677,202010) -> 3.0
a(AIT716,202010) -> 1.0
a(CEIE524,202010) -> 1.0
a(CEIE575,202010) -> 3.0
a(CEIE576,202010) -> 14.0
a(CEIE578,202010) -> 3.0
a(CEIE605,202010) -> 13.0
a(CEIE679,202010) -> 1.0
a(CEIE795,202010) -> 4.0
a(DAEN500,202010) -> 1.0


In [31]:
for i in list(b_decision_vars.values()):
    print(f"{i.name()} -> {i.SolutionValue()}")

b(AIT524,202010) -> 47.0
b(AIT542,202010) -> 47.0
b(AIT602,202010) -> 48.0
b(AIT673,202010) -> 48.0
b(AIT677,202010) -> 46.0
b(AIT716,202010) -> 48.0
b(CEIE524,202010) -> 48.0
b(CEIE575,202010) -> 46.0
b(CEIE576,202010) -> 35.0
b(CEIE578,202010) -> 46.0
b(CEIE605,202010) -> 36.0
b(CEIE679,202010) -> 48.0
b(CEIE795,202010) -> 45.0
b(DAEN500,202010) -> 48.0


In [ ]:
process_connection.execute("CREATE TABLE results_table("
                       "rec_id INTEGER PRIMARY KEY AUTOINCREMENT DEFAULT 0, "
                       "textual_context TEXT NOT NULL, "
                       "number_of_students INTEGER NOT NULL"
                       ");")
process_connection.commit()

**NOW WE CAN GET INTO THE INTERESTING STUFF**

This is for when I have more data; only focusing on what I know for certain given the total number of students available.

In [ ]:
waitlist_courses = total_courses.loc[total_courses["reg_status"] == "W",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()
dropped_courses = total_courses.loc[total_courses["reg_status"] == "D",["reg_term_code","stu_prog","crs","sect_id","reg_status"]].groupby(["reg_term_code","stu_prog","crs","sect_id"]).count().reset_index()

In [ ]:
waitlist_courses = waitlist_courses.rename(columns={'reg_status':'waitlist'})
required_courses = required_courses.rename(columns={'reg_status':'registered'})
dropped_courses = dropped_courses.rename(columns={'reg_status':'dropped'})

In [ ]:
total_courses = required_courses.merge(waitlist_courses, on=["reg_term_code", "stu_prog","crs","sect_id"], how="left").fillna(0)
total_courses = total_courses.merge(dropped_courses, on=["reg_term_code", "stu_prog","crs","sect_id"], how="left").fillna(0)